### EmbeddingBag
1. Vocab을 가져와 EmbeddingBag을 사용

In [ ]:
# 이터레이터 토큰화 함수
import pandas as pd
from konlpy.tag import Okt
from torchtext.vocab import build_vocab_from_iterator

Okt = Okt()

def yield_tokens(data_iter):
    for text in data_iter:
        # print(text)
        yield iter(Okt.nouns(text))

# next(yield_tokens(file3['text'])) # 확인함

# 파일 읽기

file_path = '../DATA/king_3.csv'
file = pd.read_csv(file_path, encoding='utf-8', sep=';')
data = file['text'].values

# vocab 생성
vocab = build_vocab_from_iterator(yield_tokens(data), specials=["<unk>"])
# vocab 확인
print('길이 :', len(vocab))
print(vocab.get_itos()[:10])

# <unk> 인덱스를 0으로 설정
vocab.set_default_index(vocab["<unk>"])

In [ ]:

# 텍스트 -> 정수 인코딩
# - lambda 함수 인스턴스 생성

text_pipeline = lambda x: vocab(tokenizer(x))
    # x를 토큰화한 후, 단어 사전에 있는 단어들을 정수로 변환

# 레이블 -> 정수 인코딩
label_pipeline = lambda x: int(x) - 1

from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# 데이터 전처리 함수
def collate_batch(batch):
    """
    batch : (label, text)
    """
    label_list, text_list, offsets = [], [], [0]  # offsets : 텍스트 길이

    for _label, _text in batch:
        # (1) 라벨 정수 인코딩
        label_list.append(label_pipeline(_label))  
        # (2) 텍스트 정수 인코딩
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        
        text_list.append(processed_text)  # 텍스트 리스트에 추가
        offsets.append(processed_text.size(0))  # 텍스트 길이 추가

    # tensor 변환
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)

    return label_list.to(device), text_list.to(device), offsets.to(device)

# 데이터 로더 생성
dataloader = DataLoader(
    train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch
)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
class TextModel(nn.Module):
    """
    텍스트 분류 모델 정의; nn.EmbeddingBag 사용 연습
    """
    
    def __init__(self, VOCAB_SIZE, EMBED_DIM, HIDDEN_SIZE, NUM_CLASS):
        """_summary_ : 모델 초기화 함수
        
        Args:
            VOCAB_SIZE (int): 어휘 사전 크기
            EMBED_DIM (int): 임베딩 차원 (단어 벡터 차원)
            HIDDEN_SIZE (int): 은닉층 크기
            NUM_CLASS (int): 분류 클래스 개수
        """
        super().__init__()
        self.embedding = nn.EmbeddingBag(VOCAB_SIZE, EMBED_DIM, sparse=False)
        self.fc = nn.Linear(EMBED_DIM, NUM_CLASS)
        self.init_weights()     # 가중치 초기화 : 학습 전에 사용
        
    def init_weights(self):
        """_summary_ : 가중치 초기화 함수
        
        .unform_() : 균등 분포로 weight 초기화
        .zero_() : 0으로 초기화
        -initrange ~ initrange 사이의 값으로 초기화
        """
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

### DOC2VEC 도 해볼까?